# Import packages

In [1]:
# read csv
import pandas as pd
import os
# PdfMiner
import glob
import numpy as np
from io import StringIO
from io import BytesIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
# Token Vectorization
from langdetect import detect 
import fasttext.util
import fasttext
########################################
from random import random
from numpy import array
from numpy import cumsum
########################################
import regex as re  
import string
import re
########################################
from datetime import datetime
import collections
########################################
# Keras imports for ML Model
import tensorflow as tf
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Conv2D, MaxPooling2D, LSTM, Bidirectional
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D, Embedding
from keras.utils import np_utils
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score

Using TensorFlow backend.


In [3]:
print(tf.__version__)

2.3.0


## Load Model

In [4]:
os.getcwd()

'C:\\Meine Dateien\\Kit Wiing Master\\SS 20\\Seminar - Knowledge Discovery\\Cyrillic_Script_Papers_KD'

In [6]:
# Load the trained model
# CHANGE PATH!
model = tf.keras.models.load_model("D:/_Training_on_all/Adam-Epochs-25-dropout-0-4-dim-100-Recall.h5")

In [7]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 1000, 200)         164800    
_________________________________________________________________
dense (Dense)                (None, 1000, 100)         20100     
_________________________________________________________________
dropout (Dropout)            (None, 1000, 100)         0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000, 4)           404       
Total params: 185,304
Trainable params: 185,304
Non-trainable params: 0
_________________________________________________________________
None


## Load FastText

In [8]:
### load FastText Modules
# CHANGE PATH!
# path_to_bins ='C:/_KDDM_Seminar/python/fastText/_bins/'
path_to_bins ='D:/fastText/' # Igor
### import RU fasttext model in reduced version (100 dim.)
ft_ru = fasttext.load_model(path_to_bins+'cc.ru.100.bin')
### import UK fasttext model in reduced version (100 dim.)
ft_uk = fasttext.load_model(path_to_bins+'cc.uk.100.bin')
### import BG fasttext model in reduced version (100 dim.)
ft_bg = fasttext.load_model(path_to_bins+'cc.bg.100.bin')

## Functions

In [10]:
### Funktion zum entfernen von Zeilenumbrüchen
def removePassage(my_str):
    my_str1 = re.sub("\\\\ud", " ", my_str)
    my_str2 = re.sub("\\\\n", " ", my_str1)
    return(my_str2)

### Funktion zum parsen von PDF[nur erste Seite] zu String Format
def extract_page_one(path):
    output_string = StringIO()
    
    with open(path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        interpreter.process_page(list(PDFPage.create_pages(doc))[0])
        return(output_string)
    
### Funktion zum entfernen von 'Arrays' aus Autoren
def removeAutor(my_str):
    my_str1 = re.sub("\['", "", my_str)
    my_str2 = re.sub("'\]", "", my_str1)
    my_str3 = re.sub("'", "", my_str2)
    return(my_str3)

### Funktion zum Vergrößern der Input & Output Vektoren um NEWLINES
def add_newlines(Tokens,Real_Tokens,y_final):
    y_final_REAL = []
    k = 0
    m = 0
    for i in range(len(Tokens)):
        if k == 0:
            j=i
        else:
            if m == 0:
                j = k+1
            else:
                j = m+1
        if Tokens[i] == Real_Tokens[j] : # Wenn Tokens gelich sind, dann übernehme y_final_REAL
            y_final_REAL.append(y_final[i])
            m = j
        else:
            for k in range(j,len(Real_Tokens)): # Sonst gehe die Real_Tokens durch bis match vorhanden

                if Real_Tokens[k] == 'NEWLINE':
                    y_final_REAL.append('Sonstiges')
                
#                 elif Real_Tokens[k] not in Tokens:
#                     y_final_REAL.append('Sonstiges')

                else:
                    y_final_REAL.append(y_final[i])
                    m=k
                    break

    RealTokens_final = Real_Tokens[:len(y_final_REAL)]
    
    index_title = [i for i, e in enumerate(y_final_REAL) if e == 'I-title']
    end_title = max(index_title)
    
    ### lable NEWLINES im Titel als "I-title"        
    for i in range(len(RealTokens_final)):
        if RealTokens_final[i]=='NEWLINE':
            if (y_final_REAL[i+1] =='I-title' or end_title>i) and y_final_REAL[i-1] in ('B-title','I-title'):
                y_final_REAL[i] = 'I-title'
    ## Kann passieren, dass im Titel mehrere NEWLINES aufeinander folgen. Daher end_title>i um festzustellen, ob iwann später noch ein Titel Label kommt        
                
    return(RealTokens_final,y_final_REAL)

#Create Word Vector representation
def detect_and_vectorize(tokens_sequence): #### input is the tokens list of ONE PAPER e.g. ergebnis_tokens[2]
    
    tokens_vectorized = []
    lang = detect(' '.join(tokens_sequence))
    
    if (lang == 'ru'):
        for i in range(len(tokens_sequence)):
            tokens_vectorized.append(np.float16(ft_ru.get_word_vector(tokens_sequence[i])))
                
    elif (lang == 'bg'):
        for i in range(len(tokens_sequence)):
            tokens_vectorized.append(np.float16(ft_bg.get_word_vector(tokens_sequence[i])))
                
    else:  ## assume language == uk
        for i in range(len(tokens_sequence)):
            tokens_vectorized.append(np.float16(ft_uk.get_word_vector(tokens_sequence[i])))
    
    while len(tokens_vectorized)<1000:
        tokens_vectorized.append(np.zeros(100))
      
    if len(tokens_vectorized)>1000:
        del tokens_vectorized[1000:] 

    return np.array(tokens_vectorized)

### Additional Features
punctuations = '''!()[]{};:'"\<>/?@#$%^&*«»_–~.,-'''

def compute_additional_features(tokens_sequence): #### input is the tokens list of ONE PAPER e.g. ergebnis_tokens[2]
    
    tokens = tokens_sequence
    feature_upper = []
    feature_capitalized = []
    feature_autor_format = []
    feature_punctation = []
    feature_newline = []
    feature_array = []
    
    while len(tokens)<1000:
        tokens.append(str(0))
    if len(tokens)>1000:
        del tokens[1000:] 
    #print(tokens)    
    for i in range(len(tokens)):
        if tokens[i] !='NEWLINE':
            if str(tokens[i]).isupper():
                feature_upper.append(1)

            else:
                feature_upper.append(0)
        else: 
            feature_upper.append(0)

        if tokens[i] !='NEWLINE':
            if str(tokens[i][0]).isupper():
                feature_capitalized.append(1)

            else:
                feature_capitalized.append(0)
        else: 
            feature_capitalized.append(0)

        if tokens[i] !='NEWLINE':
            if re.match('.\.',str(tokens[i])) != None and str(tokens[i]).isupper():
                feature_autor_format.append(1)

            else:
                feature_autor_format.append(0)
        else: 
            feature_autor_format.append(0)

        if tokens[i] !='NEWLINE':
            if any((c in punctuations) for c in str(tokens[i])):
                feature_punctation.append(1)
            else:
                feature_punctation.append(0)
        else: 
            feature_punctation.append(0)
                
        if tokens[i] =='NEWLINE':
            feature_newline.append(1)
        else: 
            feature_newline.append(0)
    df = pd.DataFrame(list(zip(feature_upper, feature_capitalized,feature_autor_format ,feature_punctation,feature_newline)))  
    feature_array = df.to_numpy(copy=True)
    
    return np.array(feature_array)

## Import

In [15]:
# CHANGE PATH!
path_meta = 'D:/_final_selection_16478/'
path_papers = "D:/_final_selection_16478/PDFs_15553"

In [13]:
### Import Meta Data

df_meta = pd.read_csv(path_meta + 'final_items_15553.csv', sep = ',')#  , encoding= 'utf-16')
# df_meta.drop('Unnamed: 0' , axis = 1 , inplace=True)
df_meta

,coreId,title,authors
0,11083759,Середовище проведення освітніх вебінарів «WIP ...,"['Богачков, Ю.М.']"
1,11083794,Комплексна підготовка дистанційних матеріалів,"['Богачков, Ю.М.', 'Царенко, В.О.']"
2,11083801,Метод аналізу елементарних доменів для покраще...,"['Богачков, Ю.М.', 'Ухань, П.С.', 'Полобюк, Т...."
3,11083807,Інструментальні засоби профорієнтації,"['Богачков, Ю.М.', 'Буров, О.Ю.', 'Бельська, Н..."
4,11083814,Експеримент «Дистанційне навчання школярів”: п...,"['Богачков, Ю.М.']"
...,...,...,...
15548,95313001,Підігрів тротуарів за допомогою теплових мереж,"['Семенюк, К.А.']"
15549,95313006,Шлакозоловидалення як спосіб підвищення енерго...,"['Бондар, Г.Ю.']"
15550,95313010,Підвищення ефективності роботи каміну,"['Півень, Д.В.']"
15551,95313012,Технологія поквартирного обліку теплової енергії,"['Горбачьова, А.О.']"


In [14]:
### Import Indices of the Test set
# CHANGE PATH!
df_test_id = pd.read_csv("D:\_Training_on_all\Test_Indices_Adam-Epochs-25-dropout-0-4-dim-100.csv", sep = ',',header=None)#  , encoding= 'utf-16')
df_test_id = df_test_id.T
df_test_id

,0
0,9648
1,4767
2,2852
3,9840
4,14077
...,...
3067,14181
3068,8077
3069,3046
3070,8585


In [16]:
pdf_15553 = os.listdir(path_papers)

all_pdfs = []
for elem in df_test_id[0]:
    all_pdfs.append(pdf_15553[elem])
    
all_pdfs

['Core_ID_12086585.pdf',
 'Core_ID_11336429.pdf',
 'Core_ID_11328295.pdf',
 'Core_ID_12086885.pdf',
 'Core_ID_42032087.pdf',
 'Core_ID_13358587.pdf',
 'Core_ID_11331737.pdf',
 'Core_ID_33757727.pdf',
 'Core_ID_11331317.pdf',
 'Core_ID_11323170.pdf',
 'Core_ID_33757130.pdf',
 'Core_ID_33757381.pdf',
 'Core_ID_11330822.pdf',
 'Core_ID_11323945.pdf',
 'Core_ID_14040333.pdf',
 'Core_ID_11313804.pdf',
 'Core_ID_141489441.pdf',
 'Core_ID_12081119.pdf',
 'Core_ID_14717226.pdf',
 'Core_ID_11312421.pdf',
 'Core_ID_12080756.pdf',
 'Core_ID_33756976.pdf',
 'Core_ID_11322559.pdf',
 'Core_ID_145231757.pdf',
 'Core_ID_12086129.pdf',
 'Core_ID_18597558.pdf',
 'Core_ID_11336146.pdf',
 'Core_ID_18597581.pdf',
 'Core_ID_11314402.pdf',
 'Core_ID_12082709.pdf',
 'Core_ID_11320638.pdf',
 'Core_ID_11312248.pdf',
 'Core_ID_11336285.pdf',
 'Core_ID_11312444.pdf',
 'Core_ID_11312847.pdf',
 'Core_ID_11337481.pdf',
 'Core_ID_12080678.pdf',
 'Core_ID_42032080.pdf',
 'Core_ID_12084279.pdf',
 'Core_ID_12081194.pdf'

In [17]:
len(all_pdfs)

3072

In [21]:
files_coreId = []
files_paths = []
for i in all_pdfs:
    files_paths.append(path_papers +"\\" + i)
    files_coreId.append(re.sub(".pdf","",re.sub("Core_ID_","",i)))

In [22]:
### extract text 

all_pdf_text = [] 
start_time = datetime.now()
for i in range(len(files_paths)):

    all_pdf_text.append(extract_page_one(files_paths[i]).getvalue())
    print(str((i/len(files_paths))*100)+'%')
    
    
end_time = datetime.now()

0.0%
0.03255208333333333%
0.06510416666666666%
0.09765625%
0.13020833333333331%
0.16276041666666669%
0.1953125%
0.22786458333333334%
0.26041666666666663%
0.29296875%
0.32552083333333337%
0.35807291666666663%
0.390625%
0.4231770833333333%
0.4557291666666667%
0.48828125%
0.5208333333333333%
0.5533854166666667%
0.5859375%
0.6184895833333333%
0.6510416666666667%
0.68359375%
0.7161458333333333%
0.7486979166666667%
0.78125%
0.8138020833333334%
0.8463541666666666%
0.87890625%
0.9114583333333334%
0.9440104166666666%
0.9765625%
1.0091145833333335%
1.0416666666666665%
1.07421875%
1.1067708333333335%
1.1393229166666665%
1.171875%
1.2044270833333335%
1.2369791666666665%
1.26953125%
1.3020833333333335%
1.3346354166666665%
1.3671875%
1.3997395833333335%
1.4322916666666665%
1.46484375%
1.4973958333333335%
1.5299479166666665%
1.5625%
1.5950520833333333%
1.6276041666666667%
1.66015625%
1.6927083333333333%
1.7252604166666667%
1.7578125%
1.7903645833333333%
1.8229166666666667%
1.85546875%
1.8880208333333

15.885416666666666%
15.91796875%
15.950520833333334%
15.983072916666666%
16.015625%
16.048177083333336%
16.080729166666664%
16.11328125%
16.145833333333336%
16.178385416666664%
16.2109375%
16.243489583333336%
16.276041666666664%
16.30859375%
16.341145833333336%
16.373697916666664%
16.40625%
16.438802083333336%
16.471354166666664%
16.50390625%
16.536458333333336%
16.569010416666664%
16.6015625%
16.634114583333336%
16.666666666666664%
16.69921875%
16.731770833333336%
16.764322916666664%
16.796875%
16.829427083333336%
16.861979166666664%
16.89453125%
16.927083333333336%
16.959635416666664%
16.9921875%
17.024739583333336%
17.057291666666664%
17.08984375%
17.122395833333336%
17.154947916666664%
17.1875%
17.220052083333336%
17.252604166666664%
17.28515625%
17.317708333333336%
17.350260416666664%
17.3828125%
17.415364583333336%
17.447916666666664%
17.48046875%
17.513020833333336%
17.545572916666664%
17.578125%
17.610677083333336%
17.643229166666664%
17.67578125%
17.708333333333336%
17.7408854

31.282552083333332%
31.315104166666668%
31.34765625%
31.380208333333332%
31.412760416666668%
31.4453125%
31.477864583333332%
31.510416666666668%
31.54296875%
31.575520833333332%
31.608072916666668%
31.640625%
31.673177083333332%
31.705729166666668%
31.73828125%
31.770833333333332%
31.803385416666668%
31.8359375%
31.868489583333332%
31.901041666666668%
31.93359375%
31.966145833333332%
31.998697916666668%
32.03125%
32.06380208333333%
32.09635416666667%
32.12890625%
32.16145833333333%
32.19401041666667%
32.2265625%
32.25911458333333%
32.29166666666667%
32.32421875%
32.35677083333333%
32.38932291666667%
32.421875%
32.45442708333333%
32.48697916666667%
32.51953125%
32.55208333333333%
32.58463541666667%
32.6171875%
32.64973958333333%
32.68229166666667%
32.71484375%
32.74739583333333%
32.77994791666667%
32.8125%
32.84505208333333%
32.87760416666667%
32.91015625%
32.94270833333333%
32.97526041666667%
33.0078125%
33.04036458333333%
33.07291666666667%
33.10546875%
33.13802083333333%
33.170572916

47.265625%
47.29817708333333%
47.33072916666667%
47.36328125%
47.39583333333333%
47.42838541666667%
47.4609375%
47.49348958333333%
47.52604166666667%
47.55859375%
47.59114583333333%
47.62369791666667%
47.65625%
47.68880208333333%
47.72135416666667%
47.75390625%
47.78645833333333%
47.81901041666667%
47.8515625%
47.88411458333333%
47.91666666666667%
47.94921875%
47.98177083333333%
48.01432291666667%
48.046875%
48.07942708333333%
48.11197916666667%
48.14453125%
48.17708333333333%
48.20963541666667%
48.2421875%
48.27473958333333%
48.30729166666667%
48.33984375%
48.37239583333333%
48.40494791666667%
48.4375%
48.47005208333333%
48.50260416666667%
48.53515625%
48.56770833333333%
48.60026041666667%
48.6328125%
48.66536458333333%
48.69791666666667%
48.73046875%
48.76302083333333%
48.79557291666667%
48.828125%
48.86067708333333%
48.89322916666667%
48.92578125%
48.95833333333333%
48.99088541666667%
49.0234375%
49.05598958333333%
49.08854166666667%
49.12109375%
49.15364583333333%
49.18619791666667

62.79296875%
62.825520833333336%
62.858072916666664%
62.890625%
62.923177083333336%
62.955729166666664%
62.98828125%
63.020833333333336%
63.053385416666664%
63.0859375%
63.118489583333336%
63.151041666666664%
63.18359375%
63.216145833333336%
63.248697916666664%
63.28125%
63.313802083333336%
63.346354166666664%
63.37890625%
63.411458333333336%
63.444010416666664%
63.4765625%
63.509114583333336%
63.541666666666664%
63.57421875%
63.606770833333336%
63.639322916666664%
63.671875%
63.704427083333336%
63.736979166666664%
63.76953125%
63.802083333333336%
63.834635416666664%
63.8671875%
63.899739583333336%
63.932291666666664%
63.96484375%
63.997395833333336%
64.02994791666666%
64.0625%
64.09505208333334%
64.12760416666666%
64.16015625%
64.19270833333334%
64.22526041666666%
64.2578125%
64.29036458333334%
64.32291666666666%
64.35546875%
64.38802083333334%
64.42057291666666%
64.453125%
64.48567708333334%
64.51822916666666%
64.55078125%
64.58333333333334%
64.61588541666666%
64.6484375%
64.68098958

78.77604166666666%
78.80859375%
78.84114583333334%
78.87369791666666%
78.90625%
78.93880208333334%
78.97135416666666%
79.00390625%
79.03645833333334%
79.06901041666666%
79.1015625%
79.13411458333334%
79.16666666666666%
79.19921875%
79.23177083333334%
79.26432291666666%
79.296875%
79.32942708333334%
79.36197916666666%
79.39453125%
79.42708333333334%
79.45963541666666%
79.4921875%
79.52473958333334%
79.55729166666666%
79.58984375%
79.62239583333334%
79.65494791666666%
79.6875%
79.72005208333334%
79.75260416666666%
79.78515625%
79.81770833333334%
79.85026041666666%
79.8828125%
79.91536458333334%
79.94791666666666%
79.98046875%
80.01302083333334%
80.04557291666666%
80.078125%
80.11067708333334%
80.14322916666666%
80.17578125%
80.20833333333334%
80.24088541666666%
80.2734375%
80.30598958333334%
80.33854166666666%
80.37109375%
80.40364583333334%
80.43619791666666%
80.46875%
80.50130208333334%
80.53385416666666%
80.56640625%
80.59895833333334%
80.63151041666666%
80.6640625%
80.69661458333334%

94.79166666666666%
94.82421875%
94.85677083333334%
94.88932291666666%
94.921875%
94.95442708333334%
94.98697916666666%
95.01953125%
95.05208333333334%
95.08463541666666%
95.1171875%
95.14973958333334%
95.18229166666666%
95.21484375%
95.24739583333334%
95.27994791666666%
95.3125%
95.34505208333334%
95.37760416666666%
95.41015625%
95.44270833333334%
95.47526041666666%
95.5078125%
95.54036458333334%
95.57291666666666%
95.60546875%
95.63802083333334%
95.67057291666666%
95.703125%
95.73567708333334%
95.76822916666666%
95.80078125%
95.83333333333334%
95.86588541666666%
95.8984375%
95.93098958333334%
95.96354166666666%
95.99609375%
96.02864583333334%
96.06119791666666%
96.09375%
96.12630208333334%
96.15885416666666%
96.19140625%
96.22395833333334%
96.25651041666666%
96.2890625%
96.32161458333334%
96.35416666666666%
96.38671875%
96.41927083333334%
96.45182291666666%
96.484375%
96.51692708333334%
96.54947916666666%
96.58203125%
96.61458333333334%
96.64713541666666%
96.6796875%
96.71223958333334

In [23]:
time_delta = (end_time - start_time)
seconds_all = time_delta.total_seconds()
minutes_all = seconds_all/60
print('Time duration for text extractions: '+str(minutes_all)+' min.')

Time duration for text extractions: 9.473010133333334 min.


In [24]:
### Define files_core_id

files_core_id = files_coreId

In [25]:
### get all titles from meta data with core_ids of fulltext
titles = []
for i in range(len(files_core_id)):
    index = df_meta.index[df_meta['coreId'] == int(files_core_id[i])].tolist()
    if index == []:
        titles.append('Keine Meta Daten gefunden')
    else: 
        index = index[0]
        title_pdf  = df_meta.loc[index,'title']
        titles.append(title_pdf)
len(titles)

3072

In [26]:
### Get autor for the PDF´s
######## PROBLEM: verschiedene Schreibweisen Meta <-> PDF
autors = []
for i in range(len(files_core_id)):
    index = df_meta.index[df_meta['coreId'] == int(files_core_id[i])].tolist()
    index = index[0]
    autor_pdf  = df_meta.loc[index,'authors']

    autor_pdf = removeAutor(autor_pdf).split(",")
    for j in range(len(autor_pdf)):
        autor_pdf[j] = ' '.join(autor_pdf[j].split()) ## Entferne überflüssige Whitespaces (auch am Anfang)
        
    autors.append(autor_pdf)
len(autors)

3072

## LABELING

In [27]:
### Funktion, die die Daten labelt 

kein_autor = []
kein_titel = []
error_papers = []
ergebnis_tokens = []
ergebnis_label = []
anzahl_papers = len(files_core_id)

for paper in range(anzahl_papers):
    title = ' '.join(removePassage(titles[paper]).split()).lower() # Remove excces Whitespace & to lowercase
    title = re.sub("\(","\(",title) # () as non-regex string
    title = re.sub("\)","\)",title)

    title_index = re.search(title, ' '.join(all_pdf_text[paper].split()).lower())
    #print('TITEL:  ' + title)
    
    if title_index==None:
        kein_titel.append(files_core_id[paper])
    else:
        try:
            Text_pdf_0 = ' '.join(all_pdf_text[paper].split())
            if title_index.start()==0:
                teil_B = ""
            else:
                teil_B = Text_pdf_0[0:title_index.start()-1]
            teil_T = Text_pdf_0[title_index.start():title_index.end()]
            teil_E = Text_pdf_0[title_index.end()+1:len(Text_pdf_0)]

            y_teil1 = np.repeat('Sonstiges',len(teil_B.split()))
            y_teil2 = np.append(['B-title'],np.repeat('I-title',len(teil_T.split())-1))
            y_teil3 = np.repeat('Sonstiges',len(teil_E.split()))

            y_final = np.concatenate((y_teil1, y_teil2 , y_teil3), axis=None)

            ##### Get Text
            all_pdf_text1 = re.sub("\\n"," NEWLINE ",all_pdf_text[paper])
            Text_pdf_0_NL = ' '.join(all_pdf_text1.split())

            Tokens = Text_pdf_0.split()
            Labels = y_final
            Real_Tokens = Text_pdf_0_NL.split()

            autors_surname = []
            for i in range(len(autors[paper])):
                if i % 2 == 0:
                    autors_surname.append(autors[paper][i])

            autors_surname_lower = []
            for i in range(len(autors_surname)):
                autors_surname_lower.append(autors_surname[i].lower())
        
            if re.match('.\.',autors[paper][1]) == None:
                autors_forename = []
                for i in range(len(autors[paper])):
                    if i % 2 == 1:
                        autors_forename.append(autors[paper][i].split())

                autors_forename = list(np.concatenate((autors_forename), axis=None))
                autors_forename_lower = []
                for i in range(len(autors_forename)):
                    autors_forename_lower.append(autors_forename[i].lower())

                autors_surname_lower = list(np.concatenate((autors_forename_lower,autors_surname_lower), axis=None))
    
            Tokens = all_pdf_text[paper].split()
            Tokens_final_lower = []
            for i in range(len(Tokens)):
                Tokens_final_lower.append(Tokens[i].lower())

            vec_autor = []
            for token in Tokens_final_lower:
                line = any(word in token for word in autors_surname_lower)
                vec_autor.append(line)

            index_autor = [i for i, e in enumerate(vec_autor) if e == True]

            if len(index_autor)>(len(autors_surname_lower)):
                diff = len(index_autor) - len(autors_surname_lower)
                dist = []
                for j in range(len(index_autor)):
                    dist.append(abs(index_autor[j]-np.where(y_final=="B-title")[0][0]))

                dict1 = dict(zip(dist , index_autor))

                dist.sort(reverse = True)

                for k in range(len(dist[0:diff])):
                    vec_autor[dict1[dist[0:diff][k]]] = False

            for i in range(len(y_final)):
                if vec_autor[i] == True:
                    y_final[i] = 'autor'

            if True not in vec_autor:
                kein_autor.append(files_core_id[paper])
                
            if re.match('.\.',autors[paper][1]) != None:

                index_autor_true = [i for i, e in enumerate(vec_autor) if e == True]

                for w in range(len(index_autor_true)):
                    index = index_autor_true[w]
                    for t in range(index - 4,index + 4):
                        if re.match('.\.',Tokens_final_lower[t]) != None and Tokens[t].isupper():
                            y_final[t] = 'autor'

            RealTokens_final = add_newlines(Tokens,Real_Tokens,y_final)[0]
            y_final_REAL = add_newlines(Tokens,Real_Tokens,y_final)[1]

            ergebnis_label.append(y_final_REAL)
            ergebnis_tokens.append(RealTokens_final)
        except:
            error_papers.append(files_core_id[paper])
            

In [28]:
kein_autor

[]

In [29]:
kein_titel

[]

In [30]:
error_papers

[]

# Preparation for ML Processing

In [31]:
### get vector representation for all PDF content and cast to float 16-bit for better RAM efficiency

all_pdfs_vectorized= []
lauf=0
start_time = datetime.now()

for i in range(len(ergebnis_tokens)):
    all_pdfs_vectorized.append(np.float16(detect_and_vectorize(ergebnis_tokens[i])))
#     all_pdfs_vectorized.append(detect_and_vectorize(ergebnis_tokens[i]))
    lauf+=1
    if lauf%500==0:
        print(str((lauf/len(ergebnis_tokens))*100)+'%')
    
end_time = datetime.now()
time_delta = (end_time - start_time)
seconds_all = time_delta.total_seconds()
minutes_all = seconds_all/60
print('Time duration for text vectorization: '+str(minutes_all)+' min.')

16.276041666666664%
32.55208333333333%
48.828125%
65.10416666666666%
81.38020833333334%
97.65625%
Time duration for text vectorization: 1.2430744333333335 min.


In [32]:
### replace newline vectors with a uniform representation (not 3 diff. vectors for bg, uk and ru)

newline_vec_uk = np.float16(ft_uk.get_word_vector('NEWLINE'))
newline_ru = np.float16(ft_ru.get_word_vector('NEWLINE'))
newline_bg = np.float16(ft_bg.get_word_vector('NEWLINE'))
counter_replaced_ru = 0
counter_replaced_bg = 0

for i in range(len(all_pdfs_vectorized)):
    
    for j in range(len(all_pdfs_vectorized[i])):
        
        compare_array=all_pdfs_vectorized[i][j].copy()
        
        if (compare_array == newline_ru).all():
            all_pdfs_vectorized[i][j] = newline_vec_uk
            counter_replaced_ru +=1
            
        elif (compare_array == newline_bg).all():
            all_pdfs_vectorized[i][j] = newline_vec_uk
            counter_replaced_bg +=1
            
print('RU Replacement Count: '+str(counter_replaced_ru))
print('BG Replacement Count: '+str(counter_replaced_bg))

RU Replacement Count: 59979
BG Replacement Count: 222


In [33]:
### add feature vectors to word vectors

all_pdfs_vectorized_features = []

for i in range(len(all_pdfs_vectorized)):
    pdf_vectorized_features = []
    features_in_pdf = compute_additional_features(ergebnis_tokens[i])
    
    for j in range(len(all_pdfs_vectorized[i])):
        pdf_vectorized_features.append(np.append(np.float16(all_pdfs_vectorized[i][j]),np.float16(features_in_pdf[j])))
        
    all_pdfs_vectorized_features.append(np.array(pdf_vectorized_features))

all_pdfs_vectorized_features=np.array((all_pdfs_vectorized_features))
all_pdfs_vectorized_features.shape

(3072, 1000, 105)

In [34]:
all_pdfs_vectorized_features[0].shape

(1000, 105)

In [35]:
### fit labels to the same length as the 1000 character input PDF string

labels_categorized = []
label_dict = {'Sonstiges':0,'B-title':1,'I-title':2,'autor':3} ## before 0 instead of None

for i in range(len(ergebnis_label)):
    transformed = [label_dict.get(n, n) for n in ergebnis_label[i]]
    
    while len(transformed)<1000:
        transformed.append(np.zeros(1))
    
    if len(transformed)>1000:
        del transformed[1000:] 
    
    categorical = np_utils.to_categorical(transformed)
    labels_categorized.append(np.array(categorical))

labels_categorized = np.array(labels_categorized)
labels_categorized.shape

(3072, 1000, 4)

In [36]:
test_files = files_core_id

predictions_all = []
for i in range(len(test_files)):
    test_exemplar_index=i
    prediction = model.predict(all_pdfs_vectorized_features[test_exemplar_index:test_exemplar_index+1])
    predictions_all.append(prediction)

In [37]:
results_all = []
for j in range(len(test_files)):
    results=[]
    for i in range(1000):
        results.append(np.argmax(predictions_all[j][0][i]))
    results_all.append(results)

In [38]:

real_all = []
for j in range(len(test_files)):
    real = []
    test_exemplar_index=j
    for i in range(1000):
        real.append(np.argmax(labels_categorized[test_exemplar_index][i]))
    real_all.append(real)

In [39]:
print(len(results_all))

print(len(real_all))

3072
3072


# Evaluation

## Accuracy

In [50]:
# Model 1

acc = []
acc_100 = []
for i in range(len(test_files)):
    acc.append(accuracy_score(real_all[i], results_all[i]))
    if real_all[i]== results_all[i]:
        acc_100.append(1)
        
print("Das Model hat eine Accuracy von: " + str(np.mean(acc)*100)+ " %")
print("Komplett richtig klassifizierte papers: " + str(len(acc_100)/len(acc)*100) + " %")


Das Model hat eine Accuracy von: 99.22672526041666 %
Komplett richtig klassifizierte papers: 7.421875 %


## Macro Jaccard Score

In [51]:
Jaccard_score = []
for p in range(len(real_all)):
    jac = jaccard_score(real_all[p], results_all[p] , average="macro")
    Jaccard_score.append(jac)
    
print("Das Model hat einen Macro Jaccard Score von: " + str(np.mean(Jaccard_score)*100)+ " %")

Das Model hat einen Macro Jaccard Score von: 67.04886367445796 %


## Macro F1 Score

In [52]:
F1_score = []
for p in range(len(real_all)):
    jac = f1_score(real_all[p], results_all[p] , average="macro")
    F1_score.append(jac)

print("Das Model hat einen Macro F1 Score von: " + str(np.mean(F1_score)*100)+ " %")

Das Model hat einen Macro F1 Score von: 71.28173552721996 %
